Please install QOKit from source. If you have issues with urllib3, downgrade it after QOKit install by running `pip install urllib3==1.26.6`

First, we check that all the simulators are loaded properly

In [2]:
from qokit.fur import get_available_simulator_names

print(get_available_simulator_names("x"))

['c', 'python']


The output should be `['gpu', 'c', 'python']`. If the `'c'` simulator is missing, you can compile it manually by running `make -C qokit/fur/c/csim/src/` in the home directory of QOKit

In [3]:
import numpy as np
from tqdm import tqdm
import networkx as nx
import timeit 
from qokit.qaoa_objective_labs import get_qaoa_labs_objective
from qokit.qaoa_objective_maxcut import get_qaoa_maxcut_objective
from qokit.qaoa_objective_sk import get_qaoa_sk_objective
from qokit.qaoa_objective import get_qaoa_objective

In [4]:
from qokit.utils import precompute_energies

Note that `f_labs` requires a precomputed diagonal for higher `N`. 
You can precompute the diagonal once using 
```
from qokit.labs import negative_merit_factor_from_bitstring
ens = precompute_energies(negative_merit_factor_from_bitstring, N)
outpath = f"../qokit/assets/precomputed_merit_factors/precomputed_energies_{N}.npy"
np.save(outpath, ens, allow_pickle=False)
```
and saving it on disc under `qokit/assets/precomputed_merit_factors`. Note that precomputation can take a while.

In [9]:
# number of qubits
for N in [8, 10, 12]:
    print(f"N={N}")
    # QAOA depth
    p = 6

    theta = np.random.uniform(0,1,2*p)
    G = nx.complete_graph(N) #, seed=42)
    J = np.random.randn(N, N)
    J = (J + J.T)/2

    for edge in G.edges:
        G.edges[edge[0], edge[1]]['weight'] = 1 #J[edge[0], edge[1]]

    # Function initialization may not be fast
    f_maxcut = get_qaoa_maxcut_objective(N, p, G)
    f_sk = get_qaoa_sk_objective(N, p, G)
    f_labs = get_qaoa_labs_objective(N, p)
    
    print(f_maxcut(theta), f_sk(theta))

    # Function evaluation is fast
    for f, label in [(f_labs, "LABS"), (f_maxcut, "MaxCut"), (f_sk, "SK")]:
        f(theta) # do not count the first evaluation
        times = []
        for _ in tqdm(range(10)):
            start = timeit.default_timer()
            f(theta)
            end = timeit.default_timer()
            times.append(end-start)
        print(f"\t{label} finished in {np.mean(times):.4f} on average, min: {np.min(times):.4f}, max: {np.max(times):.4f}")

N=8
-11.969406178348745 2.406151532752335


100%|██████████| 10/10 [00:00<00:00, 2966.27it/s]


	LABS finished in 0.0003 on average, min: 0.0003, max: 0.0005


100%|██████████| 10/10 [00:00<00:00, 3140.15it/s]


	MaxCut finished in 0.0003 on average, min: 0.0003, max: 0.0003


...

	LABS finished in 0.0011 on average, min: 0.0008, max: 0.0035


100%|██████████| 10/10 [00:00<00:00, 95.48it/s]


	MaxCut finished in 0.0104 on average, min: 0.0008, max: 0.0908


100%|██████████| 10/10 [00:00<00:00, 107.87it/s]

	SK finished in 0.0092 on average, min: 0.0008, max: 0.0831


This is what I measured on `g4dn.2xlarge` (NVIDIA T4 GPU) as the time to evaluate `f_labs(theta)` and `f_maxcut(theta)` (commit `f6f6f565`):

```
N=24
	LABS finished in 0.1823 on average, min: 0.1802, max: 0.1902
	MaxCut finished in 0.1676 on average, min: 0.1637, max: 0.1758
N=26
	LABS finished in 0.8143 on average, min: 0.8102, max: 0.8229
	MaxCut finished in 0.7606 on average, min: 0.7571, max: 0.7692
N=28
	LABS finished in 3.2480 on average, min: 3.2361, max: 3.2598
	MaxCut finished in 2.9858 on average, min: 2.9793, max: 2.9949
```

Same benchmark on `g5.2xlarge` (NVIDIA A10G):

```
N=24
	LABS finished in 0.0821 on average, min: 0.0806, max: 0.0885
	MaxCut finished in 0.0773 on average, min: 0.0759, max: 0.0836
N=26
	LABS finished in 0.3509 on average, min: 0.3481, max: 0.3601
	MaxCut finished in 0.3308 on average, min: 0.3289, max: 0.3393
N=28
	LABS finished in 1.3889 on average, min: 1.3856, max: 1.3953
	MaxCut finished in 1.3112 on average, min: 1.3088, max: 1.3201
```

On large memory CPU nodes, you can run with more qubits using the `'c'` simulator. 

In [ ]:
# number of qubits
for N in [12, 16, 20, 24, 28, 32, 33, 34]:
    print(f"N={N}")
    # QAOA depth
    p = 6

    theta = np.random.uniform(0,1,2*p)
    G = nx.random_regular_graph(4, N, seed=42)

    # Function initialization may not be fast
    f_labs = get_qaoa_labs_objective(N, p, simulator='c')

    # Function evaluation is fast
    for f, label in [(f_labs, "LABS")]:
        f(theta) # do not count the first evaluation
        times = []
        for _ in tqdm(range(3)):
            start = timeit.default_timer()
            f(theta)
            end = timeit.default_timer()
            times.append(end-start)
        print(f"For N = {N}, {label} finished in {np.mean(times):.4f} on average, min: {np.min(times):.4f}, max: {np.max(times):.4f}")

Here's what I measured on `r5.24xlarge` (Intel Xeon Platinum 8175M CPU @ 2.50GHz, 24 Cores per socket with 2 sockets, 2 threads per core, 768 GB RAM):

```
For N = 12, LABS finished in 0.0013 on average, min: 0.0012, max: 0.0016
For N = 16, LABS finished in 0.1293 on average, min: 0.1180, max: 0.1483
For N = 20, LABS finished in 0.1510 on average, min: 0.1325, max: 0.1745
For N = 24, LABS finished in 0.7461 on average, min: 0.7184, max: 0.7760
For N = 28, LABS finished in 11.4725 on average, min: 11.3852, max: 11.6067
For N = 32, LABS finished in 234.7204 on average, min: 234.3182, max: 235.0045
For N = 33, LABS finished in 482.1637 on average, min: 476.6924, max: 485.6158
For N = 34, LABS finished in 958.0517 on average, min: 946.1425, max: 968.6004
```

In [10]:
import numpy as np

In [14]:
n=3

In [15]:
x = np.random.randn(n, n)

In [16]:
x

array([[ 0.48929965,  0.24325887, -0.08034045],
       [-1.05869945, -0.6424519 , -1.92976591],
       [-0.98251804,  0.43707901, -0.51245069]])

In [21]:
np.fill_diagonal(x, 1)

In [22]:
x

array([[ 1.        ,  0.24325887, -0.08034045],
       [-1.05869945,  1.        , -1.92976591],
       [-0.98251804,  0.43707901,  1.        ]])